# Saving and Loading ModelInducer() Configs

You can save your induction configurations and load them in another workspace to replicate a setup. 

In this notebook we will save and load an inducer config and replicate the setup in a different workspace.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    umnai_env_vars.update({
        'DATABRICKS_HOST',
        'DATABRICKS_TOKEN',
    })

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths According to Your Environment
Now we will set the workspace path and the experiment path automatically. They will be set to a local path if you are using a local machine environment or to a Databricks path if you are using a Databricks environment.

## Install Databricks SDK

This checks if you are running on Databricks and installs their SDK if you are.

In [3]:
import os
if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    %pip install databricks-sdk --quiet

If you are on Databricks, you can select whether you would like the workspace to be created in the shared area (available to all users in your account) or in your personal user account area. You can ignore this if you are running on a local environment.

In [4]:
# Set to 1 if you want to use shared or 0 to use personal user account area.
USE_SHARED_WORKSPACE = 1 

## Set Paths
Next the workspace and experiment paths are set automatically. In this case we will create two workspaces so we need to create both paths.

In [5]:
import os

EXP_NAME = 'saveloadConfig_adult_income'
EXP2_NAME = 'saveloadConfig_new_adult_income'

if os.environ.get('DATABRICKS_RUNTIME_VERSION') is not None:
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()

    # # For a Databricks Environment
    WS_PATH = '/dbfs/FileStore/workspaces/'+EXP_NAME
    WS2_PATH = '/dbfs/FileStore/workspaces/'+EXP2_NAME
    if USE_SHARED_WORKSPACE:
        EXP_PREFIX = f'/Shared/experiments/'
    else:
        USERNAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
        EXP_PREFIX = f'/Users/{USERNAME}/experiments/'
    w.workspace.mkdirs(EXP_PREFIX)
    EXP_PATH = EXP_PREFIX + EXP_NAME
    EXP2_PATH = EXP_PREFIX + EXP2_NAME
else:
    # For a Local Machine Environment
    WS_PATH = 'resources/workspaces/'+EXP_NAME
    EXP_PATH = EXP_NAME
    WS2_PATH = 'resources/workspaces/'+EXP2_NAME
    EXP2_PATH = EXP2_NAME

# Create or Open a Hybrid Intelligence Workspace
Workspaces are used by the Hybrid Intelligence framework to organize your data and models together in one place.

In [6]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path = WS_PATH,
    experiment = EXP_PATH
)

ws # Prints workspace details to confirm created/opened

# Create ModelInducer Config

In [7]:
from umnai.induction.inducer import ModelInducer

# Simple model inducer using fast execution parameters
model_inducer = ModelInducer(
    max_interactions=3,
    max_interaction_degree=2,
    max_polynomial_degree=2,
    trials=2,
    estimators=2,
    batch_size=512,
    iterations=2,
)

#### You can now use this Model Inducer to induce a Model.

# Save ModelInducer Config

In [8]:
# Save ModelInducer config
model_inducer.save_config(f'{WS_PATH}/{"resources/configs/inducer_config.json"}')

# Reuse the Config in a New Workspace
Next we will use the JSON file we just saved to recreate the induction setup in a new workspace.

# Create a New Hybrid Intelligence Workspace

In [9]:
# Close the open workspace as you can only have one workspace open at a time
ws.close()

# On a local machine environment:
ws2 = Workspace.open(
    path = WS2_PATH,
    experiment = EXP2_PATH
)

ws2 # Prints workspace details to confirm created/opened

# Load ModelInducer Config
Load the config from the JSON file saved from another workspace.

In [10]:
# Load Model Inducer config from previous workspace
model_inducer_replicated = ModelInducer.load_config(file_path=f'{WS_PATH}/{"resources/configs/inducer_config.json"}')

Check parameters are same as those saved earlier:

In [11]:
# Confirm parameters as same:
print("max_interactions : ", model_inducer_replicated.max_interactions)
print("max_interaction_degree : ", model_inducer_replicated.max_interaction_degree)
print("max_polynomial_degree : ", model_inducer_replicated.max_polynomial_degree)
print("trials : ", model_inducer_replicated.trials)
print("estimators : ", model_inducer_replicated.estimators)
print("batch_size : ", model_inducer_replicated.batch_size)
print("iterations : ", model_inducer_replicated.iterations)

max_interactions :  3
max_interaction_degree :  2
max_polynomial_degree :  2
trials :  2
estimators :  2
batch_size :  512
iterations :  2


#### You can now use the replicated model inducer to induce a model.